## Data

In [1]:
import numpy as np
import tensorflow as tf

from hopfield import DenseRecon, DiscreteTimeHopfieldLayer, soft_sign
from mnist import load_mnist

tf.random.set_seed(42)

tf.keras.backend.clear_session()

print(tf.__version__)

2.3.0


In [2]:
# IMAGE_SIZE = (32, 32)
IMAGE_SIZE = (8, 8)  # XXX: test!
BINARIZE = True
# BINARIZE = False

In [3]:
(x_train, _), _ = load_mnist(image_size=IMAGE_SIZE, binarize=BINARIZE)

## Model

In [4]:
class Padding(tf.keras.layers.Layer):

    def __init__(self, num_pads, pad_value, **kwargs):
        super().__init__(**kwargs)
        self.num_pads = num_pads
        if isinstance(pad_value, (list, tuple)):
            self.pad_value = tuple(map(float, pad_value))
        else:
            self.pad_value = float(pad_value)

    def get_config(self):
        config = super().get_config()
        config['num_pads'] = self.num_pads
        config['pad_value'] = self.pad_value
        return config

    def build(self, batch_input_shape):
        self._padding_shape = batch_input_shape[:-1] + [self.num_pads]
        super().build(batch_input_shape)

    def call(self, x):
        padding_shape = tf.concat(
            [tf.shape(x)[:-1], tf.constant([self.num_pads])], axis=0)
        if isinstance(self.pad_value, tuple):
            minval, maxval = self.pad_value
            pads = sign(
                tf.random.uniform(
                    padding_shape, minval, maxval, dtype=x.dtype))
        else:
            pads = self.pad_value * tf.ones(padding_shape)
        y = tf.concat([x, pads], axis=-1)
        return y


def sign(x):
    y = tf.where(x > 0, 1, -1)
    y = tf.cast(y, x.dtype)
    return y

In [5]:
def create_model():
    model = tf.keras.Sequential([
        DiscreteTimeHopfieldLayer(
            DenseRecon(),
            max_steps=20,
            reg_factor=1),
    ])
    model.compile(optimizer='adam')
    return model

In [6]:
def create_model_by_repeating(num_repeat=1):
    model = tf.keras.Sequential([
        tf.keras.layers.RepeatVector(num_repeat),
        tf.keras.layers.Flatten(),
        DiscreteTimeHopfieldLayer(
            DenseRecon(),
            max_steps=20,
            reg_factor=1),
        tf.keras.layers.Reshape([num_repeat, -1]),
        tf.keras.layers.Lambda(
            lambda x: soft_sign(tf.reduce_mean(x, axis=-2))),
    ])
    model.compile(optimizer='adam')
    return model

In [7]:
def create_model_by_padding(num_pads, pad_value):
    model = tf.keras.Sequential([
        Padding(num_pads, pad_value),
        DiscreteTimeHopfieldLayer(
            DenseRecon(),
            max_steps=20,
            reg_factor=1),
        tf.keras.layers.Lambda(
            lambda x: x[..., :(IMAGE_SIZE[0] * IMAGE_SIZE[1])]),
    ])
    model.compile(optimizer='adam')
    return model

In [8]:
# X = x_train[:100].numpy()
X = x_train.numpy().reshape([-1, 1024])[:100]

ds0 = tf.data.Dataset.from_tensor_slices(X)
ds = ds0.shuffle(10000).repeat(10000).batch(128)
model = create_model()
# model = create_model_by_padding(num_pads=(64 * 15), pad_value=-1)
model.fit(ds)

7813/7813 [==============================] - 161s 21ms/step - loss: 0.0190


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
discrete_time_hopfield_layer (None, 1024)              1049601   
Total params: 1,049,601
Trainable params: 1,049,600
Non-trainable params: 1
_________________________________________________________________


In [10]:
# noised_X = X + np.random.normal(size=X.shape) * 0.3
noised_X = np.where(np.random.random(size=X.shape) < 0.3, -X, X)
recon_X = model.predict(noised_X)

for layer in model.layers:
    try:
        print('Relax steps:', layer.final_step.numpy())
    except AttributeError:
        pass

orig_err = noised_X - X
err = recon_X - X
print(f'{np.quantile(np.abs(orig_err), 0.99)} => '
      f'{np.quantile(np.abs(err), 0.99)}')

Relax steps: 6
2.0 => 0.0


In [11]:
X, recon_X

(array([[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]], dtype=float32),
 array([[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.]], dtype=float32))

## Conclusions



### Methods of Dimensional Extension

* repeating: ×
* padding by given value: ×
* padding by random value: ×
* input grouping: √